In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from os import listdir
from os.path import isfile, join
import seaborn as sns
from multiprocessing import Pool

In [19]:
oil = pd.read_csv("data/oil.csv")
stores = pd.read_csv("data/stores.csv")
holidays_events = pd.read_csv("data/holidays_events.csv")
transactions = pd.read_csv("data/transactions.csv")
items = pd.read_csv("data/items.csv")
mainTrain = pd.read_pickle("trainArray1Mil.pickle")
mainTest = pd.read_pickle("testArray1Mil.pickle")

In [20]:
mainTrain["dcoilwtico"] = mainTrain["dcoilwtico"].fillna(method="pad")
mainTest["dcoilwtico"] = mainTest["dcoilwtico"].fillna(method="pad")
mainTrain.head()
# print(mainTrain.isnull().sum())

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico,family,class,perishable,month,year,day
0,22000000,2014-04-07,39,215352,38.0,False,Cuenca,Azuay,B,6,1447,100.43,GROCERY I,1016,0,4,2014,7
1,22000001,2014-04-07,39,215356,1.0,False,Cuenca,Azuay,B,6,1447,100.43,GROCERY I,1016,0,4,2014,7
2,22000002,2014-04-07,39,215370,4.0,False,Cuenca,Azuay,B,6,1447,100.43,BREAD/BAKERY,2718,1,4,2014,7
3,22000003,2014-04-07,39,215896,11.0,False,Cuenca,Azuay,B,6,1447,100.43,GROCERY I,1044,0,4,2014,7
4,22000004,2014-04-07,39,218728,4.0,False,Cuenca,Azuay,B,6,1447,100.43,CLEANING,3044,0,4,2014,7


In [ ]:
#catgorical columns
store_nbrCol =  tf.feature_column.categorical_column_with_identity("store_nbr", 54)
onpromotionCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("onpromotion", hash_bucket_size=2), dimension=3)
typeCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("type", hash_bucket_size=5), dimension=3)
familyCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("family", hash_bucket_size=33), dimension=3)
# classCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("class", hash_bucket_size=354), dimension=3)
# perishableCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("perishable", hash_bucket_size=2), dimension=3)
# monthCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("month", hash_bucket_size=12), dimension=3)
mainTrain.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico,family,class,perishable,month,year,day
0,22000000,2014-04-07,39,3958,38.0,False,19,8,2,7,1447,100.43,15,131,0,4,2014,7
1,22000001,2014-04-07,39,703,1.0,False,19,8,2,7,1447,100.43,15,131,0,4,2014,7
2,22000002,2014-04-07,39,1124,4.0,False,19,8,2,7,1447,100.43,5,93,1,4,2014,7
3,22000003,2014-04-07,39,2024,11.0,False,19,8,2,7,1447,100.43,15,163,0,4,2014,7
4,22000004,2014-04-07,39,264,4.0,False,19,8,2,7,1447,100.43,24,225,0,4,2014,7


In [23]:
CATEGORICAL_COLUMNS =["store_nbr"]# ["month", "perishable", "onpromotion", "type", 'class']
CONTINUOUS_COLUMNS = ["transactions", "dcoilwtico", "day"]
LABEL_COLUMN = "unit_sales"

def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = {**continuous_cols, **categorical_cols}
  # Converts the label column into a constant Tensor.
  label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
  return feature_cols, label

def train_input_fn():
  return input_fn(mainTrain)

def eval_input_fn():
    return input_fn(mainTest)

In [24]:
dayCol = tf.contrib.layers.real_valued_column("day")
oilCol = tf.contrib.layers.real_valued_column("dcoilwtico")
transactionsCol = tf.contrib.layers.real_valued_column("transactions")

In [25]:
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNRegressor(feature_columns=[dayCol, oilCol, store_nbrCol],
                                   optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
#       l1_regularization_strength=0.01,
        l2_regularization_strength=0.01
        
    ),
  hidden_units=[64, 32, 16])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_master': '', '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, '_save_checkpoints_secs': 600, '_environment': 'local', '_tf_random_seed': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_model_dir': 'C:\\Users\\gadge\\AppData\\Local\\Temp\\tmpmpmrqmge', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015F825470F0>, '_task_id': 0, '_is_chief': True, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_session_config': None}


In [27]:
m.fit(input_fn=train_input_fn, steps=10000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Restoring parameters from C:\Users\gadge\AppData\Local\Temp\tmpmpmrqmge\model.ckpt-10000


INFO:tensorflow:Saving checkpoints for 10001 into C:\Users\gadge\AppData\Local\Temp\tmpmpmrqmge\model.ckpt.


INFO:tensorflow:loss = 272.919, step = 10001


...

INFO:tensorflow:loss = 272.918, step = 12501 (29.258 sec)


INFO:tensorflow:global_step/sec: 3.41444


INFO:tensorflow:loss = 272.918, step = 12601 (29.287 sec)


INFO:tensorflow:global_step/sec: 3.40954


INFO:tensorflow:loss = 272.918, step = 12701 (29.329 sec)
